# Iterative phase estimation using `qdk-chemistry`

This notebook provides an example of `qdk-chemistry` functionality through an end-to-end workflow estimating the ground state energy of a multi-configurational quantum chemistry system.  This is one example of a wide range of functionality in `qdk-chemistry`. Please see <https://github.com/microsoft/qdk-chemistry> for the full documentation.

In [ ]:
# Load frequently used external packages
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd

# Reduce logging output for demo
from qdk_chemistry.utils import Logger
Logger.set_global_level(Logger.LogLevel.off)

## Loading the stretched N<sub>2</sub> structure

This example uses a *stretched* N<sub>2</sub> molecule, which introduces multi-reference character in the wavefunction. The structure is loaded from a [XYZ-format](https://en.wikipedia.org/wiki/XYZ_file_format) file.

In [ ]:
from qdk_chemistry.data import Structure

# Stretched N2 structure at 1.270025 Å bond length
structure = Structure.from_xyz_file(Path("data/stretched_n2.structure.xyz"))

## Generating and optimizing the molecular orbitals

This step performs a Hartree-Fock (HF) SCF calculation to generate an approximate initial wavefunction and ground-state energy guess.

In [ ]:
from qdk_chemistry.algorithms import create

# Perform an SCF calculation, returning the energy and wavefunction
scf_solver = create("scf_solver")
E_hf, wfn_hf = scf_solver.run(structure, charge=0, spin_multiplicity=1, basis_or_guess="cc-pvdz")

Unlike the basis functions, canonical molecular orbitals from SCF calculations are often delocalized over the entire molecule. As an example, we use the MP2 natural orbital localization method in `qdk-chemistry` to generate orbitals that tend to yield more chemically meaningful representations.

The resulting molecular orbitals will be used in subsequent steps for active space selection and multi-configuration calculations.

In [ ]:
from qdk_chemistry.utils import compute_valence_space_parameters

# Reduce the number of orbitals
num_val_e, num_val_o = compute_valence_space_parameters(wfn_hf, charge=0)
active_space_selector = create(
    "active_space_selector",
    "qdk_valence",
    num_active_electrons=num_val_e,
    num_active_orbitals=num_val_o
)
valence_wf = active_space_selector.run(wfn_hf)

# Localize the orbitals
localizer = create("orbital_localizer", "qdk_mp2_natural_orbitals")
valence_indices = valence_wf.get_orbitals().get_active_space_indices()
loc_wfn = localizer.run(valence_wf, *valence_indices)
print("Localized orbitals:\n", loc_wfn.get_orbitals().get_summary())

## Optimizing problem size with active space selection

Active space selection focuses the quantum calculation on a subset of the electrons and orbitals in the system.

This example uses `qdk_autocas_eos`, an automated entropy-based active-space selection method to identify strongly
correlated orbitals.

In [ ]:
# Construct a Hamiltonian from the localized orbitals
hamiltonian_constructor = create("hamiltonian_constructor")
loc_orbitals = loc_wfn.get_orbitals()
loc_hamiltonian = hamiltonian_constructor.run(loc_orbitals)
num_alpha_electrons, num_beta_electrons = loc_wfn.get_active_num_electrons()

# Compute the selected configuration interaction wavefunction
macis_mc = create(
    "multi_configuration_calculator", "macis_asci", calculate_one_rdm=True, calculate_two_rdm=True,
    )
_, wfn_sci = macis_mc.run(loc_hamiltonian, num_alpha_electrons, num_beta_electrons)

# Optimize the problem with autoCAS-EOS active space selection
autocas = create("active_space_selector", "qdk_autocas_eos")
autocas_wfn = autocas.run(wfn_sci)
indices, _ = autocas_wfn.get_orbitals().get_active_space_indices()
print(f"autoCAS-EOS selected {len(indices)} of {num_val_o} orbitals for the active space:  indices={list(indices)}")

The next step constructs the active-space Hamiltonian and computes a multi-configuration wavefunction for the selected
active space.
This step also provides a reference energy for the active space system that can be used to benchmark the iQPE result.

In [ ]:
# Construct the active space Hamiltonian
hamiltonian_constructor = create("hamiltonian_constructor")
refined_orbitals = autocas_wfn.get_orbitals()
active_hamiltonian = hamiltonian_constructor.run(refined_orbitals)

# Calculate the exact wavefunction and energy with CASCI
alpha_electrons, beta_electrons = autocas_wfn.get_active_num_electrons()
mc = create("multi_configuration_calculator", "macis_cas")
e_cas, wfn_cas = mc.run(active_hamiltonian, alpha_electrons, beta_electrons)
print(f"Active space system energy: {e_cas:.3f} Hartree")

Visualizing the selected active space is an important step to ensure that the selected orbitals are chemically meaningful.
This step uses visualization tools built into `qdk` to display the selected active space orbitals.

In [ ]:
from qdk.widgets import MoleculeViewer
from qdk_chemistry.utils.cubegen import generate_cubefiles_from_orbitals

active_orbitals = wfn_cas.get_orbitals()

# Generate cube files for the active orbitals
cube_data = generate_cubefiles_from_orbitals(
    orbitals=active_orbitals,
    grid_size=(40, 40, 40),
    margin=10.0,
    indices=active_orbitals.get_active_space_indices()[0],
    label_maker=lambda p: f"{'occupied' if p < 20 else 'virtual'}_{p + 1:04d}"
)

# Visualize the molecular orbitals together with the structure
MoleculeViewer(molecule_data=structure.to_xyz(), cube_data=cube_data)

## Optimizing trial wavefunction loading onto a quantum computer

The multi-configuration wavefunction in the active space can serve as the trial state for the iQPE algorithm.
However, this information needs to be loaded as a state on a quantum computer.

The amount of data loaded onto the quantum computer can be optimized by exploiting the sparsity of the wavefunction.
This step identifies the dominant configurations in the wavefunction using visualization tools provided by `qdk`.

In [ ]:
from qdk.widgets import Histogram

# Plot top configuration weights from the CASCI wavefunction
print(f"Total configurations in the CASCI wavefunction:  {len(wfn_cas.get_active_determinants())}")
print("Plotting the configurations by weight.")
top_configurations = wfn_cas.get_top_determinants()
display(
    Histogram(
        bar_values={k.to_string(): np.abs(v)**2 for k, v in top_configurations.items()}, 
        items="top-25", 
        sort="high-to-low",
        )
    )

To run quantum phase estimation, we need to prepare an initial trial state for the calculation.
In this example, we will take the first two terms of the multi-configuration wavefunction, add a small amount of noise, and check their overlap with the full wavefunction.

Choosing fewer terms still gives us good overlap in the trial state, and also illustrates QPE output with imperfect starting information.

In [ ]:
from utils.qpe_utils import prepare_2_dets_trial_state

# Prepare a trial state with two determinants (and noise). Compute its overlap with the CASCI wavefunction.
wfn_trial, fidelity = prepare_2_dets_trial_state(wfn_cas)
print(f"Overlap of trial state with CASCI wavefunction: {fidelity:.2%}")

# Generate a plot of the configurations in the trial wavefunction
configurations = wfn_trial.get_top_determinants()
display(
    Histogram(
        bar_values={k.to_string(): np.abs(v)**2 for k, v in configurations.items()},
        sort="high-to-low",
    )
)

There are many ways to "load" this state onto a quantum computer.
This example uses a popular method as a basis for comparison with our chemistry-aware optimized approach.
Circuit statistics are shown and the circuit is visualized using built-in `qdk` functions.

In [ ]:
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state using the regular isometry method (Qiskit)
state_prep = create("state_prep", "regular_isometry")
regular_isometry_circuit = state_prep.run(wfn_trial)

# Visualize the regular isometry circuit
display(Circuit(regular_isometry_circuit.get_qsharp()))

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(
    estimate(regular_isometry_circuit.get_qasm()).logical_counts.items(),
    columns=['Logical Estimate', 'Counts']
)
display(df)

The popular approach for state preparation requires a larger number of operations with numerous fine rotations.
However, `qdk-chemistry` provides optimized state preparation methods that exploit the structure of chemistry wavefunctions to reduce the number of operations and improve noise resilience.

In [ ]:
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state via GF2+X sparse isometry
state_prep = create("state_prep", "sparse_isometry_gf2x")
sparse_isometry_circuit = state_prep.run(wfn_trial)

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(
    estimate(sparse_isometry_circuit.get_qasm()).logical_counts.items(),
    columns=['Logical Estimate', 'Counts']
)
display(df)

# Visualize the sparse isometry circuit, idle and classical qubits are removed
display(Circuit(sparse_isometry_circuit.get_qsharp()))

## Estimating the ground state energy with iterative quantum phase estimation

Kitaev-style iterative quantum phase estimation (iQPE) estimates an eigenphase of the time-evolution operator $U = e^{-iHt}$ using one ancilla qubit and a sequence of controlled-$U^{2^k}$ applications.
 
Each iteration measures one bit of the phase (from most-significant to least-significant) and uses phase feedback to refine the estimate.

In [ ]:
# Set up parameters for iQPE
M_PRECISION = 10
T_TIME = 0.7
SHOTS_PER_ITERATION = 3
SIMULATOR_SEED = 42

The classical Hamiltonian for the active space must be mapped to a qubit Hamiltonian that can be measured on a quantum computer.
The Jordan-Wigner transformation is a popular mapping that is used in this example.

In [ ]:
# Prepare the qubit-mapped Hamiltonian
qubit_mapper = create("qubit_mapper", algorithm_name="qiskit", encoding="jordan-wigner")
qubit_hamiltonian = qubit_mapper.run(active_hamiltonian)
print("Qubit Hamiltonian:\n", qubit_hamiltonian.get_summary())

The circuit for iQPE consists of initial trial state preparation followed by multiple controlled time-evolution operations.
This cell visualizes the one iteration of the iQPE circuit in QASM format using built-in `qdk` visualization tools.

In [ ]:
from qdk.openqasm import circuit

# Use factory methods to create the iQPE algorithm components
evolution_builder = create("time_evolution_builder", "trotter")
circuit_mapper = create("controlled_evolution_circuit_mapper", "pauli_sequence")
iqpe = create("phase_estimation", "iterative", num_bits=M_PRECISION, evolution_time=T_TIME)

# Generate the iQPE iteration circuit for a specific iteration (3rd from last)
iqpe_iter_circuit = iqpe.create_iteration_circuit(
    state_preparation=sparse_isometry_circuit,
    qubit_hamiltonian=qubit_hamiltonian,
    evolution_builder=evolution_builder,
    circuit_mapper=circuit_mapper,
    iteration=M_PRECISION-3,
    total_iterations=M_PRECISION,
)

# Visualize the iQPE iteration circuit
display(Circuit(circuit(iqpe_iter_circuit.get_qasm())))

This real-time example performs a single-trial low-precision iQPE run on the `qdk` full state simulator.

In [ ]:
# Execute the iQPE algorithm for a single trial (l) with low precision
iqpe_low = create("phase_estimation", "iterative", num_bits=6, evolution_time=T_TIME)
circuit_executor = create("circuit_executor", "qdk_full_state_simulator")
result = iqpe_low.run(
    state_preparation=sparse_isometry_circuit,
    qubit_hamiltonian=qubit_hamiltonian,
    circuit_executor=circuit_executor,
    evolution_builder=evolution_builder,
    circuit_mapper=circuit_mapper,
)
print(list(result.branching))
print(result.raw_energy + active_hamiltonian.get_core_energy() - e_cas)

raise NotImplementedError("WIP - stopped here")

# Pick out the most common energy estimate from the results
estimated_energy, _ = Counter(
    [result.resolved_energy for result in results]
).most_common(1)[0]

# Summarize the QPE results
print(f"QPE Results of {M_PRECISION} bit precision:")
print(f"Reference CASCI energy: {e_cas:.6f} Hartree")
print(f"Total energy from phase estimation: {estimated_energy + active_hamiltonian.get_core_energy():.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy + active_hamiltonian.get_core_energy() - e_cas):.3e} Hartree")


The above cell used low precision for a real-time example.
However, chemical accuracy typically requires higher precision.
The following cell performs a higher-precision iQPE run using the same simulator.

In [ ]:
# Large number of trials with results previously calculated and saved
from qdk_chemistry.data import QpeResult
from utils.qpe_utils import run_iqpe

NUM_TRIALS = 10
RESULTS_DIR = Path(
    f"results_iqpe/precision_{M_PRECISION}/time_{T_TIME}/num_trials_{NUM_TRIALS}"
)

# Run iQPE if results do not already exist
if not RESULTS_DIR.exists():
    results = run_iqpe(
        qubit_hamiltonian,
        state_prep_circuit,
        time=T_TIME,
        precision=M_PRECISION,
        shots=SHOTS_PER_ITERATION,
        seed=SIMULATOR_SEED,
        reference_energy=e_cas - active_hamiltonian.get_core_energy(),
        trials=NUM_TRIALS,
        output_dir=RESULTS_DIR,
    )


For a system with noise or an imperfect trial state, multiple trials of iQPE are needed to obtain a reliable estimate of the ground state energy.
This estimate is typically taken as the most frequently observed energy from multiple trials ("majority vote").

In [ ]:
# Load the results
results_loaded = []
for json_file in RESULTS_DIR.glob("*qpe_result.json"):
    result = QpeResult.from_json_file(json_file)
    results_loaded.append(result)

# Count the energy values
energy_counts = Counter(
    [
        result.resolved_energy + active_hamiltonian.get_core_energy()
        for result in results_loaded
    ]
)
print(f"QPE Results of {M_PRECISION} bit precision from {NUM_TRIALS} trials:")
display(pd.DataFrame(energy_counts.items(), columns=['Energy (Hartree)', 'Counts']))

# Use the most frequently observed energy across all trials as the overall estimate
estimated_energy, _ = energy_counts.most_common(1)[0]



The iQPE energy estimate accuracy is useful for benchmarking the impact of precision, evolution time, and other parameters on the final result.
The following cell summarizes energy errors from the multiple trials.

In [ ]:
# Print summary of results
print(f"Reference CASCI energy: {e_cas:.6f} Hartree")
print(f"Total energy from phase estimation: {estimated_energy:.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy - e_cas):.3e} Hartree")

# Summarize the energy errors
energy_errors = {
    qpe_e - e_cas: count
    for qpe_e, count in sorted(energy_counts.items())
}

# Plot distribution of energy differences
print("Energy difference (Hartree) distribution:")
display(
    Histogram(
        bar_values={f"{err:.3e}": count for err, count in energy_errors.items()}
        )
    )